```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Implicit Equal Weights Particle Filter

This notebook implements prototyping and example/demo of the Implicit Equal Weights Particle Filter (IEWPF).


## Set environment

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from scipy.special import lambertw

import os
import sys

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')
matplotlib.rcParams['contour.negative_linestyle'] = 'solid'

#Import our simulator
from SWESimulators import CDKLM16, PlotHelper, Common, IPythonMagic

from SWESimulators import BathymetryAndICs as BC
from SWESimulators import OceanStateNoise
from SWESimulators import OceanNoiseEnsemble
from SWESimulators import BaseOceanStateEnsemble
from SWESimulators import DataAssimilationUtils as dautils
from SWESimulators import IEWPFOcean


In [ ]:
%cuda_context_handler gpu_ctx

# Testing IEWPF

Here, we make a test of the entire IEWPF algorithm applied to a suitable test case.

In [ ]:
# DEFINE PARAMETERS

#Coriolis well balanced reconstruction scheme
nx = 40
ny = 40

dx = 4.0
dy = 4.0

dt = 0.05*3
g = 9.81
r = 0.0

f = 0.05
beta = 0.0

ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
boundaryConditions = Common.BoundaryConditions(2,2,2,2)

# Define which cell index which has lower left corner as position (0,0)
x_zero_ref = 2
y_zero_ref = 2

dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
dataShapeHi = (ny + ghosts[0]+ghosts[2]+1, 
             nx + ghosts[1]+ghosts[3]+1)

eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
eta0_extra = np.zeros(dataShape, dtype=np.float32, order='C')
hv0 = np.zeros(dataShape, dtype=np.float32, order='C');
hu0 = np.zeros(dataShape, dtype=np.float32, order='C');
waterDepth = 1.0
Hi = np.ones(dataShapeHi, dtype=np.float32, order='C')*waterDepth

# Add disturbance:
initOption = 3
if initOption == 1:
    # Original initial conditions
    rel_grid_size = nx*1.0/dx
    BC.addBump(eta0, nx, ny, dx, dy, 0.3, 0.5, 0.05*rel_grid_size, validDomain)
    eta0 = eta0*0.3
    BC.addBump(eta0, nx, ny, dx, dy, 0.7, 0.3, 0.10*rel_grid_size, validDomain)
    eta0 = eta0*(-1.3)
    BC.addBump(eta0, nx, ny, dx, dy, 0.15, 0.8, 0.03*rel_grid_size, validDomain)
    eta0 = eta0*1.0
    BC.addBump(eta0, nx, ny, dx, dy, 0.6, 0.75, 0.06*rel_grid_size, validDomain)
    BC.addBump(eta0, nx, ny, dx, dy, 0.2, 0.2, 0.01*rel_grid_size, validDomain)
    eta0 = eta0*(-0.03)
    BC.addBump(eta0_extra, nx, ny, dx, dy, 0.5, 0.5, 0.4*rel_grid_size, validDomain)
    eta0 = eta0 + 0.02*eta0_extra
    BC.initializeBalancedVelocityField(eta0, Hi, hu0, hv0, f, beta, g, nx, ny, dx ,dy, ghosts)
    eta0 = eta0*0.5
elif initOption == 2:
    # Initial conditions used for the SIR filter
    rel_grid_size = nx*1.0/dx
    BC.addBump(eta0, nx, ny, dx, dy, 0.3, 0.5, 0.05*rel_grid_size, validDomain)
    eta0 = eta0*0.3
    BC.addBump(eta0, nx, ny, dx, dy, 0.7, 0.3, 0.10*rel_grid_size, validDomain)
    eta0 = eta0*(-1.3)
    BC.addBump(eta0, nx, ny, dx, dy, 0.15, 0.8, 0.03*rel_grid_size, validDomain)
    eta0 = eta0*1.0
    BC.addBump(eta0, nx, ny, dx, dy, 0.6, 0.75, 0.06*rel_grid_size, validDomain)
    BC.addBump(eta0, nx, ny, dx, dy, 0.2, 0.2, 0.01*rel_grid_size, validDomain)
    eta0 = eta0*(-0.03)
    BC.addBump(eta0_extra, nx, ny, dx, dy, 0.5, 0.5, 0.4*rel_grid_size, validDomain)
    eta0 = eta0 + 0.02*eta0_extra
    BC.initializeBalancedVelocityField(eta0, Hi, hu0, hv0, f, beta, g, nx, ny, dx ,dy, ghosts)
    eta0 = eta0*0.5
elif initOption == 3:
    # Initial conditions random - see further down!
    pass
    

if 'sim' in globals():
    sim.cleanUp()
if 'ensemble' in globals():
    ensemble.cleanUp()
if 'iewpfOcean' in globals():
    iewpfOcean.cleanUp()
    
q0 = 0.5*dt*f/(g*waterDepth)
print "q0: ", q0
print "[f, g, H, dt]", [f, g, waterDepth, dt]
print "f/gH: ", f/(g*waterDepth)
print "gH/f: ", g*waterDepth/f

reload(CDKLM16)
reload(BaseOceanStateEnsemble)
reload(OceanNoiseEnsemble)
reload(PlotHelper)
reload(dautils)
reload(IEWPFOcean)

sim = CDKLM16.CDKLM16(gpu_ctx, eta0, hu0, hv0, Hi, \
                      nx, ny, dx, dy, dt, g, f, r, \
                      boundary_conditions=boundaryConditions, \
                      write_netcdf=False, \
                      small_scale_perturbation=True, \
                      small_scale_perturbation_amplitude=q0)
if initOption == 3:
    sim.perturbState(q0_scale=50)
    
ensemble_size = 10

ensemble = OceanNoiseEnsemble.OceanNoiseEnsemble(ensemble_size, gpu_ctx,  
                                                 observation_type=dautils.ObservationType.DirectUnderlyingFlow)
ensemble.setGridInfoFromSim(sim)
ensemble.setStochasticVariables(#observation_variance_factor=2.0,
                                observation_variance = 0.01**2,
                                small_scale_perturbation_amplitude=q0)
                                #initialization_variance_factor_ocean_field=50)
ensemble.init(driftersPerOceanModel=4)

max_dt = ensemble.findLargestPossibleTimeStep()
print "max_dt: ", max_dt

iewpfOcean = IEWPFOcean.IEWPFOcean(ensemble, debug=False, show_errors=True)

print "Ready!"
#ensemble.plotEnsemble()

In [ ]:
with Common.Timer("Compilation") as timer:
    infoPlots = []


    #T = 200
    T = 31
    sub_t = 2*dt
    observation_iterations = range(10, 250, 20)


    GPU = True
    #GPU = False

    for i in range(T):
        t = ensemble.step(sub_t)

        for oi in observation_iterations:
            if i == oi:
                print "Enter IEWPF for observation time ", i
                if GPU:
                    iewpfOcean.iewpf(ensemble, infoPlots, i)
                else:
                    iewpfOcean.iewpf_CPU(ensemble, infoPlots, i)
                print "Successful IEWPF (hopefully)"
print("IEWPF took " + str(timer.secs))

In [ ]:
def show_figures(figs):
    for f in figs:
        dummy = plt.figure()
        new_manager = dummy.canvas.manager
        new_manager.canvas.figure = f
        f.set_canvas(new_manager.canvas)
        filename= "iewpf_20180720_figures/" + f._suptitle.get_text().replace(" ", "_").replace("=", "_") + ".png"
        print filename
        #plt.savefig(filename)
        #plt.close()
show_figures(infoPlots)
fig = ensemble.plotDistanceInfo(title="Final ensemble")
ensemble.plotEnsemble()
#plt.savefig("iewpf_20180720_figures/final_ensemble.png")


In [ ]:
filename = "None"

fig = plt.figure(figsize=(10,3))
plt.plot(ensemble.tArray, ensemble.rmseUnderDrifter_eta, label='eta')
plt.plot(ensemble.tArray, ensemble.rmseUnderDrifter_hu,  label='hu')
plt.plot(ensemble.tArray, ensemble.rmseUnderDrifter_hv,  label='hv')
plt.plot(observation_iterations, 0.0*np.ones_like(observation_iterations), 'o')
plt.title("RMSE under drifter")
plt.legend(loc=0)
plt.grid()
plt.ylim([0, 0.6])
#filename= "iewpf_20180720_figures/RMSE_under_drifter.png"
print filename
plt.savefig(filename)

fig = plt.figure(figsize=(10,3))
plt.plot(ensemble.tArray, ensemble.varianceUnderDrifter_eta, label='eta')
plt.plot(ensemble.tArray, ensemble.varianceUnderDrifter_hu,  label='hu')
plt.plot(ensemble.tArray, ensemble.varianceUnderDrifter_hv,  label='hv')
plt.plot(observation_iterations, 0.0*np.ones_like(observation_iterations), 'o')
plt.title("Variance under drifter")
plt.legend(loc=0)
plt.grid()
plt.ylim([0, 0.6])
#filename= "iewpf_20180720_figures/var_under_drifter.png"
print filename
plt.savefig(filename)

fig = plt.figure(figsize=(10,3))
plt.plot(ensemble.tArray, ensemble.rUnderDrifter_eta, label='eta')
plt.plot(ensemble.tArray, ensemble.rUnderDrifter_hu,  label='hu')
plt.plot(ensemble.tArray, ensemble.rUnderDrifter_hv,  label='hv')
plt.plot(observation_iterations, 0.0*np.ones_like(observation_iterations), 'o')
plt.title("r = var/rmse under drifter")
plt.legend(loc=0)
plt.grid()
plt.ylim([0, 5])
#filename= "iewpf_20180720_figures/r_under_drifter.png"
print filename
plt.savefig(filename)

print np.sqrt(ensemble.observation_cov[0,0])

In [ ]:
eta, hu, hv = ensemble.particles[0].download(interior_domain_only=True)

fig = plt.figure(figsize=(10,4))

ax1 = plt.subplot(1,3,1)
plt.imshow(eta, origin="lower", interpolation="None")
plt.colorbar()

ax2 = plt.subplot(1,3,2)
plt.imshow(eta, origin="lower", interpolation="None")
plt.colorbar()

ax3 = plt.subplot(1,3,3)
plt.imshow(eta, origin="lower", interpolation="None")
plt.colorbar()

#print fig
#print ax


def _markDriftersInImshow(ax):
    
    observed_drifter_positions = ensemble.observeTrueDrifters()
    for d in range(iewpfOcean.numDrifters):
        print "drifter ", d
        cell_id_x = int(np.floor(observed_drifter_positions[d,0]/iewpfOcean.dx))
        cell_id_y = int(np.floor(observed_drifter_positions[d,1]/iewpfOcean.dy))
        circ = matplotlib.patches.Circle((cell_id_x, cell_id_y), 1, fill=False)
        ax.add_patch(circ)

_markDriftersInImshow(ax1)
_markDriftersInImshow(ax3)
_markDriftersInImshow(ax2)